In [1]:
import os
# Data handling
import pandas as pd
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from glob import glob
from bs4.element import Comment
from bs4 import BeautifulSoup

# fuzzy text matching
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# nlp tools
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re

# Doc2vec modules
import gensim
from gensim.models import doc2vec
from gensim.utils import keep_vocab_item
# For tensorboard projections
#from tfmodel import Project_Model



[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
!pip freeze > requirements.txt

### Todo:
* Predict 10 closest KBs to current KB
* Translate Non english KB articles

In [3]:
#load model
model = doc2vec.Doc2Vec.load('Models/90.1.model')

In [4]:
stops = set(stopwords.words("english"))
stops.remove('how')
stops.remove('to')
morestops = ['the', 'in', 'bms', 'com', 'kb']
[stops.add(word) for word in morestops]

[None, None, None, None, None]

In [5]:
pd.set_option('display.max_columns', 500)

Import data with utf-8 encoding

In [6]:
#knowledge = pd.read_excel("Data/KB_en_html_body.xlsx", encoding='utf-8')
knowledge = pd.read_excel("kb_knowledge_Published_IT ONLY.xlsx", encoding='utf-8')
KB_list = pd.read_excel("KB List.xlsx", encoding='utf-8')

### Map of KB numbers to Article titles

In [7]:
kb_map = {}
for row in KB_list.iterrows():
    kb_map[row[1].number] = row[1].short_description
    
def kb_mapper(kb_num):
    try:        
        return kb_map[kb_num]
    except Exception:
        return 'NaN'

# Cleaning the KB article data

Many of the non-native english author groups have KBs in other languages, we can deal with this later by translating the documents into english but for now we'll exlude these articles so we can focus on the accuracy of our model.

In [8]:
knowledge_trim = knowledge[['Number','Short description', 'Meta', 'Article body', "Author Group", "Configuration Item"]]
knowledge_trim = knowledge_trim[~knowledge["Author Group"].isin(["SERVICE DESK FIELD APAC", 
                                                                 "CS APPS AND SYSTEMS ERP AS/JPN",
                                                                 "SERVICE DESK ACN SAP SUPPORT", 
                                                                 "GBO CTP GPO TE",
                                                                 "APP SUPPORT BRM FIELD CHINA"])]
knowledge_trim.columns = ['kb_number', 'title', 'tag', 'text', 'author_group', 'CI']

Removing related articles from these author groups in our test set

In [9]:
df_copy = knowledge_trim.copy()
df_copy.dropna(inplace=True)
df_copy.reset_index(drop=True, inplace=True)
pd.set_option('max_colwidth',999)

In [10]:
df_copy1 = df_copy.copy()

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def tag_parse(text):    
    soup = BeautifulSoup(str(text), 'html.parser')
    #parse \n
    text = soup.get_text().replace('\\n',' ')
    text = text.replace('$','')
    text = text.replace('-', ' ')    
    text = text.replace('_',' ')
    #PH seems like phone number
    text = text.replace('PH', '')
    #Remove Non letters
    text = re.sub("[^a-zA-Z]", " ", text)
    # Convert words to lower case and split them
    # Remove stopwords
    words = text.lower().split()
    words = [word for word in words if word not in morestops]
    words = [word for word in words if len(word) > 1]
    return words

def text_parse(text):    
    soup = BeautifulSoup(text, 'html.parser')
    #parse 
    texts = soup.findAll(text=True)#.replace('\n','').replace('\r','')
    text = filter(tag_visible, texts)
    text = [w for w in text if w not in ['\n','\xa0', 'Description', 'References and Additional Resources', 'description']]
    text = u" ".join(t.strip() for t in text)
    text = text.replace('\xa0','').replace('\r','')
    # removing html links
    text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', text)
    #Remove Non letters
    text = re.sub("[^a-zA-Z]", " ", text)
    # Convert words to lower case and split them
    words = text.lower().split()
    # Remove stopwords
    words = [word for word in words if word not in stops]
    words = [word for word in words if len(word) > 1]
    return words

def docfix(text):
    '''For parsing configuration items'''
    text = text.replace('-', '')
    text = text.replace('  ','')
    text = text.replace(' ','_')
    return text

#Parse text collumn of bad values
df_copy1['text'] = df_copy1['text'].map(text_parse)
df_copy1['tag'] = df_copy1['tag'].map(tag_parse)
df_copy1['title_parsed'] = df_copy1['title'].map(tag_parse)
df_copy1.sample(1)

,kb_number,title,tag,text,author_group,CI,title_parsed
883,KB0026976,APEX General Information and Troubleshooting,"[apex, wwp, gpm]","[overview, internally, branded, apex, also, referred, to, model, gpm, externally, hosted, salesforce, global, system, record, approved, prices, supports, price, approval, workflow, international, reference, pricing, phased, roll, beginning, may, ending, dec, phased, implementation, markets, live, time, brands, live, time, given, market, access, access, request, must, made, directly, to, director, world, wide, pricing, cecile, carbonnelle, writing, integrated, single, sign, sso, production, url, apex, test, url, apex, test, access, outside, network, requires, factor, sso, username, password, dynamic, security, code, dynamic, security, code, used, vpn, access, usability, chrome, browser, required, tier, service, desk, support, basic, connectivity, authentication, triage, issues, routed, to, tier, vitalize, group, ...]",APP SUPPORT APEX,APEX,"[apex, general, information, and, troubleshooting]"


In [11]:
def predict(text, return_rel=False, model=model):
    """
    Infers the vectors of a list of tokens then finds the documents with the closest vectors

    Parameters
    ----------
    text: string of text you'd like to run a knowledge article prediction on
    return_rel: bool, default False
        returns either first relevent article or 10 relavent articles. default is first.
    model: The model you're predicting from.

    """
    tokens = text
    new_vector = model.infer_vector(tokens)
    similar = model.docvecs.most_similar([new_vector])
    if return_rel == False:
        similar_articles = [item[0] for item in similar]        
        return similar_articles
    else:
        #print(tokens)
        similar_articles = [item for item in similar]        
        return similar_articles

In [12]:
parsed_articles = df_copy1.copy()
pd.set_option('display.max_colwidth', -1)

def kb_predict(similar_to):
    tokens = predict(similar_to, return_rel=True, model=model)
    frame = pd.DataFrame(tokens, columns=["KB", "% Related"])
    frame["KB Number"] = frame["KB"]
    frame["KB"] = '=HYPERLINK("https://bmsprod.service-now.com/nav_to.do?uri=%2Fkb_view.do%3Fsysparm_article%3D'+ frame["KB"] +'%26sysparm_stack%3D%26sysparm_view%3D","'+ frame["KB"].map(kb_mapper) +'")'
    return frame

In [13]:
for key, row in parsed_articles[parsed_articles['CI'].isin(['Dynamic Security Code'])].iterrows():    
    result_frame = kb_predict(row['text'])
    break

In [14]:
result_frame.iloc[0]["% Related"]

0.7525719404220581

In [15]:
result_frame.iloc[0]["% Related"]

0.7525719404220581

In [16]:
result_frame[result_frame["% Related"] <= result_frame.iloc[0]["% Related"]]

,KB,% Related,KB Number
0,"=HYPERLINK(""https://bmsprod.service-now.com/nav_to.do?uri=%2Fkb_view.do%3Fsysparm_article%3DKB0020625%26sysparm_stack%3D%26sysparm_view%3D"",""DYNAMIC SECURITY CODE: User requests temporary access - a temporary one time password (TOTP)"")",0.752572,KB0020625
1,"=HYPERLINK(""https://bmsprod.service-now.com/nav_to.do?uri=%2Fkb_view.do%3Fsysparm_article%3DKB0020830%26sysparm_stack%3D%26sysparm_view%3D"",""DYNAMIC SECURITY CODE: Unlock a Dynamic Security Code Device User "")",0.491394,KB0020830
2,"=HYPERLINK(""https://bmsprod.service-now.com/nav_to.do?uri=%2Fkb_view.do%3Fsysparm_article%3DKB0020641%26sysparm_stack%3D%26sysparm_view%3D"",""DYNAMIC SECURITY CODE: User is getting an error when registering or activating a token"")",0.419261,KB0020641
3,"=HYPERLINK(""https://bmsprod.service-now.com/nav_to.do?uri=%2Fkb_view.do%3Fsysparm_article%3DKB0013155%26sysparm_stack%3D%26sysparm_view%3D"",""DYNAMIC SECURITY CODE: Can't log in to VPN troubleshooting"")",0.412731,KB0013155
4,"=HYPERLINK(""https://bmsprod.service-now.com/nav_to.do?uri=%2Fkb_view.do%3Fsysparm_article%3DKB0020810%26sysparm_stack%3D%26sysparm_view%3D"",""DYNAMIC SECURITY CODE: User has found their Dynamic Security Code device and cannot register it"")",0.407093,KB0020810
5,"=HYPERLINK(""https://bmsprod.service-now.com/nav_to.do?uri=%2Fkb_view.do%3Fsysparm_article%3DKB0020854%26sysparm_stack%3D%26sysparm_view%3D"",""How to assist a user who lost or misplaced their Dynamic Security Code Device"")",0.403964,KB0020854
6,"=HYPERLINK(""https://bmsprod.service-now.com/nav_to.do?uri=%2Fkb_view.do%3Fsysparm_article%3DKB0026167%26sysparm_stack%3D%26sysparm_view%3D"",""DYNAMIC SECURITY CODE: User needs assistance with registering or activating a device"")",0.377299,KB0026167
7,"=HYPERLINK(""https://bmsprod.service-now.com/nav_to.do?uri=%2Fkb_view.do%3Fsysparm_article%3DKB0015217%26sysparm_stack%3D%26sysparm_view%3D"",""DYNAMIC SECURITY CODE: Unable to log in to an app that uses Dynamic Security Code for authentication "")",0.361713,KB0015217
8,"=HYPERLINK(""https://bmsprod.service-now.com/nav_to.do?uri=%2Fkb_view.do%3Fsysparm_article%3DKB0020806%26sysparm_stack%3D%26sysparm_view%3D"",""DYNAMIC SECURITY CODE: eSetup request for a DSC device/VPN access awaiting approval"")",0.349430,KB0020806
9,"=HYPERLINK(""https://bmsprod.service-now.com/nav_to.do?uri=%2Fkb_view.do%3Fsysparm_article%3DKB0020631%26sysparm_stack%3D%26sysparm_view%3D"",""DYNAMIC SECURITY CODE: List of Symantec reason codes "")",0.323665,KB0020631


In [17]:
result_frame["% Related"]

0    0.752572
1    0.491394
2    0.419261
3    0.412731
4    0.407093
5    0.403964
6    0.377299
7    0.361713
8    0.349430
9    0.323665
Name: % Related, dtype: float64

In [18]:
def detect_collisions(CI, gap=0.0):
    '''
    Attributes
    -------------
    str : The CI you'd like to detect collisons for
    float  : The amount of relavence as a decimal you'd like to dispaly of KB articles 
                    - ie .30 = display articles within 30% relevance from the top article
    '''
    writer = pd.ExcelWriter('Collisions/' + CI + ".xlsx")
    for key,row in parsed_articles[parsed_articles['CI'].isin([CI])].iterrows():
        result_frame = kb_predict(row['text'])
        result_frame = result_frame[result_frame["% Related"] <= result_frame.iloc[0]["% Related"]]
        # If the frame length is equal to one pass on this kb at its most likely not a collision
        frame_length = result_frame.shape[0]
        if frame_length == 1:
            continue
        result_frame.to_excel(writer, sheet_name=row['kb_number'], index=False, encoding='utf-8')    
        for i, col in enumerate(result_frame.columns):
            # find length of column i
            column_len = result_frame[col].astype(str).str.len().max()
            # Setting the length if the column header is larger
            # than the max column value length
            column_len = max(column_len, len(col)) - 140
            # set the column length
            writer.sheets[row['kb_number']].set_column(i, i, column_len)
            break
    writer.save()

In [19]:
for key,row in parsed_articles[parsed_articles['CI'].isin(['Interact - Latam and BR'])].iterrows():
    result_frame = kb_predict(row['text'])
    result_frame = result_frame[result_frame["% Related"] <= result_frame.iloc[0]["% Related"]]
result_frame

,KB,% Related,KB Number
0,"=HYPERLINK(""https://bmsprod.service-now.com/nav_to.do?uri=%2Fkb_view.do%3Fsysparm_article%3DKB0044181%26sysparm_stack%3D%26sysparm_view%3D"",""Attaching a file to a knowledge article"")",0.286153,KB0044181
1,"=HYPERLINK(""https://bmsprod.service-now.com/nav_to.do?uri=%2Fkb_view.do%3Fsysparm_article%3DKB0033740%26sysparm_stack%3D%26sysparm_view%3D"",""Reviewing, rejecting, or approving an end user knowledge article"")",0.244053,KB0033740
2,"=HYPERLINK(""https://bmsprod.service-now.com/nav_to.do?uri=%2Fkb_view.do%3Fsysparm_article%3DKB0028987%26sysparm_stack%3D%26sysparm_view%3D"",""User is unable to find the log file."")",0.243592,KB0028987
3,"=HYPERLINK(""https://bmsprod.service-now.com/nav_to.do?uri=%2Fkb_view.do%3Fsysparm_article%3DKB0033745%26sysparm_stack%3D%26sysparm_view%3D"",""Creating parent-child knowledge articles"")",0.231054,KB0033745
4,"=HYPERLINK(""https://bmsprod.service-now.com/nav_to.do?uri=%2Fkb_view.do%3Fsysparm_article%3DKB0042366%26sysparm_stack%3D%26sysparm_view%3D"",""ASM Guide to requesting access for Maximo users"")",0.226644,KB0042366
5,"=HYPERLINK(""https://bmsprod.service-now.com/nav_to.do?uri=%2Fkb_view.do%3Fsysparm_article%3DKB0028491%26sysparm_stack%3D%26sysparm_view%3D"",""How to add an IP address to a CI in the vitalize SACM CMDB"")",0.223594,KB0028491
6,"=HYPERLINK(""https://bmsprod.service-now.com/nav_to.do?uri=%2Fkb_view.do%3Fsysparm_article%3DKB0044932%26sysparm_stack%3D%26sysparm_view%3D"",""How to use MIGO_TR to perform Place in Storage Material Document in SAP"")",0.221781,KB0044932
7,"=HYPERLINK(""https://bmsprod.service-now.com/nav_to.do?uri=%2Fkb_view.do%3Fsysparm_article%3DKB0034448%26sysparm_stack%3D%26sysparm_view%3D"",""ASM: Requesting Access ARMED at BMS"")",0.218877,KB0034448
8,"=HYPERLINK(""https://bmsprod.service-now.com/nav_to.do?uri=%2Fkb_view.do%3Fsysparm_article%3DKB0072964%26sysparm_stack%3D%26sysparm_view%3D"",""vitalize Change Management reference guide"")",0.218015,KB0072964
9,"=HYPERLINK(""https://bmsprod.service-now.com/nav_to.do?uri=%2Fkb_view.do%3Fsysparm_article%3DKB0034157%26sysparm_stack%3D%26sysparm_view%3D"",""RPM CAR attachments"")",0.217333,KB0034157


In [20]:
detect_collisions('Interact - US')
detect_collisions('Interact - EMEA')
detect_collisions('Interact Events Management - LATAM')
detect_collisions('Interact Events Management - EMEA')
detect_collisions('Interact - Latam and BR')
detect_collisions('Interact - APAC')

In [21]:
detect_collisions('Interact - Latam and BR')

In [22]:
detect_collisions('Dynamic Security Code')
detect_collisions('TRAVEL--CONCUR TRAVEL')
detect_collisions('ECLIPSE CORE')
detect_collisions('Chemistry Research Electronic Laboratory  Notebook')

In [23]:
parsed_articles['CI'].unique()

array(['VITALIZE DEMAND', 'ECLIPSE CORE', 'eSlide Manager Loader',
       'Metro', 'PLANNING AND RESOURCE MANAGEMENT',
       'JAVA WEB-ENABLED INTEGRATED REVIEW', 'eSetup',
       'Outlook Email and Calendar',
       'Chemistry Research Electronic Laboratory  Notebook',
       'Compound Request', 'Print AnyWhere EMEA', 'Athena - SalesForce',
       'TIAMO', 'Whitepages', 'MOVE IT', 'Interact - APAC',
       'SALES & MKTG DATA WAREHOUSE', 'BMS ChemTools', 'BIOBOOK',
       'MANAGEMENT ANALYSIS AND REPORTING SYSTEM', 'DATA LOAD SYSTEM',
       'PRISM - CARA', 'The BMS Yammer Network',
       'Telephone and Voicemail', 'Dynamic Security Code',
       'Data Quality Monitoring', 'INVESTIGATOR PORTAL', 'GRAB',
       'Internet Explorer', 'MaxEAM - Maximo Enterprise Asset Management',
       'GLOBAL CLINICAL SITE MONITORING',
       'ELECTRONIC SUPPLEMENTAL NOTEBOOK', 'SAP-Manufacturing', 'PDLIMS',
       'Electronic Trial Master File', 'Geneious Application',
       'Successfactors LMS', 'B

In [24]:
parsed_articles[parsed_articles['CI'].isin(['Concur'])]

,kb_number,title,tag,text,author_group,CI,title_parsed


In [25]:
def highlight_matching(data, color='#cce8cf'):
    '''
    highlight if KB name is in Predicted
    '''
    attr = 'background-color: {}'.format(color)
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        actual = data[0]
        predictions = data[7]
        predict_arry = []
        for i in data:
            if actual in predictions:
                predict_arry.append(attr)
            else:
                predict_arry.append('')
                
        return predict_arry

#parsed_articles.style.apply(highlight_matching, axis=1)

In [26]:
check_desc = master_test.copy()
check_desc["Knowledge"] = check_desc.Knowledge.map(kb_mapper)
#check_desc["Predicted"] = check_desc["Predicted"].map(lambda value : value[0])
check_desc["Predicted"] = check_desc["Predicted"].map(lambda values : [kb_mapper(kb) for kb in values])
pd.set_option('display.max_rows', total)
check_desc.style.apply(highlight_matching, axis=1)

NameError: name 'master_test' is not defined

# Lookup Tools 
---

### Kb matcher

In [ ]:
@interact (similar_to='password')
def most_similar(similar_to):
    try:
        tokens = predict(similar_to, return_rel=True, model=master_model)
        frame = pd.DataFrame(tokens, columns=["KB", "% Related"])
        frame["KB"] = frame["KB"].map(kb_mapper)        
        return frame
    except Exception as e:
        print(e)
        return "Error"

### Similar word lookup

In [ ]:
@interact (similar_to='password')
def most_similar(similar_to):
    try:
        tokens = similar_to.lower().split()        
        #similar_articles = [item[0] for item in similar]        
        similar = master_model.wv.most_similar(similar_to)
        return pd.DataFrame(similar, columns=["Word", "% Related"])
    except:
        return "I can only lookup one word at a Time"#pd.DataFrame(None, columns=["Most Similar Word", "% Related"])

### Oddball - finds the least matching word